In [1]:
import pandas as pd
# import all shapes
cen_tract_all_shapes_df  = pd.read_csv('census_tract_shapefiles_all.csv')
cen_tract = pd.read_csv('food_access_research_atlas.csv')
print(cen_tract_all_shapes_df.shape)
cen_tract_all_shapes_df.head(5)

(1955546, 5)


,GEOID,PointOrder,SubPolygonId,PointLatitude,PointLongitude
0,1001020100,1,1,32.4988,-86.4820
1,1001020100,2,1,32.4915,-86.4787
2,1001020100,3,1,32.4764,-86.4812
3,1001020100,4,1,32.4660,-86.4751
4,1001020100,5,1,32.4597,-86.4753


In [2]:
print(cen_tract.shape)
cen_tract.head(5)

(72864, 147)


,CensusTract,State,County,Urban,POP2010,OHU2010,GroupQuartersFlag,NUMGQTRS,PCTGQTRS,LILATracts_1And10,...,TractSeniors,TractWhite,TractBlack,TractAsian,TractNHOPI,TractAIAN,TractOMultir,TractHispanic,TractHUNV,TractSNAP
0,1001020100,Alabama,Autauga,1,1912,693,0,0,0.000000,0,...,221,1622,217,14,0,14,45,44,26,112
1,1001020200,Alabama,Autauga,1,2170,743,0,181,0.083410,0,...,214,888,1217,5,0,5,55,75,87,202
2,1001020300,Alabama,Autauga,1,3373,1256,0,0,0.000000,0,...,439,2576,647,17,5,11,117,87,108,120
3,1001020400,Alabama,Autauga,1,4386,1722,0,0,0.000000,0,...,904,4086,193,18,4,11,74,85,19,82
4,1001020500,Alabama,Autauga,1,10766,4082,0,181,0.016812,0,...,1126,8666,1437,296,9,48,310,355,198,488


In [7]:
# Create new Impoverished column
df = cen_tract
cond_1 = df['MedianFamilyIncome'] <= 24250
df['Impoverished'] = cond_1

# keep only impoverished or food desert for all US
df = cen_tract
cond_1 = df['Impoverished'] == True
cond_2 = (df['LILATracts_1And10'] == 1) | (df['LILATracts_halfAnd10'] == 1)  | (df['LILATracts_1And20'] == 1)  | (df['LILATracts_Vehicle'] == 1)
cond_3 = ~((df['State'] == 'Alaska') | (df['State'] == 'Hawaii'))
cond_4 = ~(cen_tract['POP2010'] == 0)
df_below_income_threshold_deserts = df.loc[(cond_1 | cond_2) & cond_3 & cond_4]
print(df_below_income_threshold_deserts.shape)
df_below_income_threshold_deserts.head(5)

(23030, 148)


,GEOID,State,County,Urban,POP2010,OHU2010,GroupQuartersFlag,NUMGQTRS,PCTGQTRS,LILATracts_1And10,...,TractWhite,TractBlack,TractAsian,TractNHOPI,TractAIAN,TractOMultir,TractHispanic,TractHUNV,TractSNAP,Impoverished
6,1001020700,Alabama,Autauga,1,2891,1188,0,36,0.012452,1,...,2333,435,13,3,11,96,98,134,218,False
11,1001021100,Alabama,Autauga,0,3320,1306,0,33,0.009940,1,...,1441,1810,2,0,14,53,39,196,262,False
17,1003010600,Alabama,Baldwin,1,3325,1186,0,14,0.004210,1,...,1323,1851,3,0,30,118,92,156,311,False
37,1003011407,Alabama,Baldwin,1,4799,2161,0,4,0.000834,1,...,4525,56,36,1,26,155,57,64,143,False
40,1003011502,Alabama,Baldwin,1,10534,4330,0,0,0.000000,1,...,7818,1693,113,8,74,828,1314,245,613,False


In [8]:
# keep shapes only for Below Income Thresholds OR all food deserts
df1 = cen_tract_all_shapes_df
df2 = df_below_income_threshold_deserts
clm_list = df2['CensusTract'].values
cen_tract_desert_shapes_df = df1[df1['GEOID'].isin(clm_list)]
print(cen_tract_desert_shapes_df.shape)
cen_tract_desert_shapes_df.head(5)

(548417, 5)


,GEOID,PointOrder,SubPolygonId,PointLatitude,PointLongitude
83,1001020700,1,1,32.4252,-86.4602
84,1001020700,2,1,32.4319,-86.4630
85,1001020700,3,1,32.4422,-86.4620
86,1001020700,4,1,32.4460,-86.4647
87,1001020700,5,1,32.4478,-86.4644


In [9]:
# Save both dataframes
clmns1 = ['GEOID', 'State', 'County','POP2010','LILATracts_1And10', 'LILATracts_halfAnd10', 'LILATracts_1And20','LILATracts_Vehicle','MedianFamilyIncome','Impoverished']
clmns2 = ["lasnaphalf", "lasnaphalfshare", "lahunvhalf", "lahunvhalfshare", "lasnap1", "lasnap1share", "lahunv1", "lahunv1share", "lasnap10", "lasnap10share", "lahunv10", "lahunv10share", "lasnap20", "lasnap20share",  "lahunv20", "lahunv20share"]
# change CensusTract to GEOID
new_columns = df_below_income_threshold_deserts.columns.values
new_columns[0] = 'GEOID'
df_below_income_threshold_deserts.columns  = new_columns
df_below_income_threshold_deserts[clmns1+clmns2].to_csv('All_US_impoverished_or_deserts.csv')
cen_tract_desert_shapes_df.to_csv('Shapes_Low_Income_OR_Food_Deserts.csv')

# End of code 

In [9]:
df_below_income_threshold_deserts.columns[9:20]

Index(['LILATracts_1And10', 'LILATracts_halfAnd10', 'LILATracts_1And20',
       'LILATracts_Vehicle', 'HUNVFlag', 'LowIncomeTracts', 'PovertyRate',
       'MedianFamilyIncome', 'LA1and10', 'LAhalfand10', 'LA1and20'],
      dtype='object')

In [10]:
# Save both dataframes
clmns1 = ['GEOID', 'State', 'County','POP2010','LILATracts_1And10', 'LILATracts_halfAnd10', 'LILATracts_1And20','LILATracts_Vehicle','MedianFamilyIncome']
clmns2 = ["Income","lasnaphalf", "lasnaphalfshare", "lahunvhalf", "lahunvhalfshare", "lasnap1", "lasnap1share", "lahunv1", "lahunv1share", "lasnap10", "lasnap10share", "lahunv10", "lahunv10share", "lasnap20", "lasnap20share",  "lahunv20", "lahunv20share"]
print(clmns1)
print(clmns1+clmns2)



['GEOID', 'State', 'County', 'POP2010', 'LILATracts_1And10', 'LILATracts_halfAnd10', 'LILATracts_1And20', 'LILATracts_Vehicle', 'MedianFamilyIncome']
['GEOID', 'State', 'County', 'POP2010', 'LILATracts_1And10', 'LILATracts_halfAnd10', 'LILATracts_1And20', 'LILATracts_Vehicle', 'MedianFamilyIncome', 'Income', 'lasnaphalf', 'lasnaphalfshare', 'lahunvhalf', 'lahunvhalfshare', 'lasnap1', 'lasnap1share', 'lahunv1', 'lahunv1share', 'lasnap10', 'lasnap10share', 'lahunv10', 'lahunv10share', 'lasnap20', 'lasnap20share', 'lahunv20', 'lahunv20share']


In [5]:
# change CensusTract to GEOID
new_columns = cen_tract.columns.values
new_columns[0] = 'GEOID'
cen_tract.columns  = new_columns

# Create new Impoverished column
df = cen_tract
cond_1 = df['MedianFamilyIncome'] <= 24250
df['Impoverished'] = cond_1
# keep only impoverished and food desert for all US
df = cen_tract
cond_1 = df['Impoverished'] == True
cond_2 = (df['LILATracts_1And10'] == 1) | (df['LILATracts_halfAnd10'] == 1)  | (df['LILATracts_1And20'] == 1)  | (df['LILATracts_Vehicle'] == 1)
df_below_income_threshold_deserts = df.loc[cond_1 | cond_2]
# print(df.shape)
# df_below_income_threshold_deserts[['MedianFamilyIncome','Impoverished','LILATracts_halfAnd10','LILATracts_1And20','LILATracts_Vehicle','LILATracts_1And10']].head(5)

# Merge dataframes
# df1 = df_below_income_threshold_deserts
# df2 = cen_tract_all_shapes_df
# merged_df = pd.merge(df1, df2, on="GEOID",how="left")
clmns = ['GEOID', 'State', 'County','POP2010','LILATracts_1And10', 'LILATracts_halfAnd10', 'LILATracts_1And20','LILATracts_Vehicle','MedianFamilyIncome',]
merged_df[].to_csv('All_US_impoverished_or_deserts.csv')
print(merged_df.shape)
merged_df.head(5)

(593982, 152)


,GEOID,State,County,Urban,POP2010,OHU2010,GroupQuartersFlag,NUMGQTRS,PCTGQTRS,LILATracts_1And10,...,TractAIAN,TractOMultir,TractHispanic,TractHUNV,TractSNAP,Impoverished,PointOrder,SubPolygonId,PointLatitude,PointLongitude
0,1001020700,Alabama,Autauga,1,2891,1188,0,36,0.012452,1,...,11,96,98,134,218,False,1,1,32.4252,-86.4602
1,1001020700,Alabama,Autauga,1,2891,1188,0,36,0.012452,1,...,11,96,98,134,218,False,2,1,32.4319,-86.4630
2,1001020700,Alabama,Autauga,1,2891,1188,0,36,0.012452,1,...,11,96,98,134,218,False,3,1,32.4422,-86.4620
3,1001020700,Alabama,Autauga,1,2891,1188,0,36,0.012452,1,...,11,96,98,134,218,False,4,1,32.4460,-86.4647
4,1001020700,Alabama,Autauga,1,2891,1188,0,36,0.012452,1,...,11,96,98,134,218,False,5,1,32.4478,-86.4644


In [25]:
# change CensusTract to GEOID
new_columns = cen_tract.columns.values
new_columns[0] = 'GEOID'
cen_tract.columns  = new_columns
cen_tract.head(2)

,GEOID,State,County,Urban,POP2010,OHU2010,GroupQuartersFlag,NUMGQTRS,PCTGQTRS,LILATracts_1And10,...,TractSeniors,TractWhite,TractBlack,TractAsian,TractNHOPI,TractAIAN,TractOMultir,TractHispanic,TractHUNV,TractSNAP
0,1001020100,Alabama,Autauga,1,1912,693,0,0,0.00000,0,...,221,1622,217,14,0,14,45,44,26,112
1,1001020200,Alabama,Autauga,1,2170,743,0,181,0.08341,0,...,214,888,1217,5,0,5,55,75,87,202


In [26]:
cen_tract.shape

(72864, 147)

In [6]:
State = "IL_IN_OH"
cond1 = cen_tract["State"]=='Illinois'
cond2 = cen_tract["State"]=='Indiana'
cond3 = cen_tract["State"]=='Ohio'
cond4 = ~(cen_tract['POP2010'] == 0)
cen_tract_city = cen_tract[(cond1 | cond2 | cond3) & cond4]
#cen_tract_city = cen_tract[(cen_tract["State"]==State)]
print(cen_tract_city.shape)
print(State)
cen_tract_city.head(5)

(7565, 147)
IL_IN_OH


,GEOID,State,County,Urban,POP2010,OHU2010,GroupQuartersFlag,NUMGQTRS,PCTGQTRS,LILATracts_1And10,...,TractSeniors,TractWhite,TractBlack,TractAsian,TractNHOPI,TractAIAN,TractOMultir,TractHispanic,TractHUNV,TractSNAP
20889,17001000100,Illinois,Adams,1,4627,2115,0,23,0.004971,0,...,1074,4412,62,45,1,7,100,72,98,180
20890,17001000201,Illinois,Adams,1,1986,872,0,20,0.010070,0,...,326,1882,53,9,1,2,39,18,45,140
20891,17001000202,Illinois,Adams,1,2999,1046,0,549,0.183061,0,...,289,2688,215,14,1,9,72,62,84,227
20892,17001000400,Illinois,Adams,1,4322,1534,0,589,0.136279,0,...,890,3527,582,13,0,9,191,78,202,496
20893,17001000500,Illinois,Adams,1,2337,889,0,273,0.116816,0,...,429,1872,380,6,1,5,73,18,171,230


In [21]:
df[['MedianFamilyIncome','Impoverished']].loc[df['Impoverished'] == True]

,MedianFamilyIncome,Impoverished
20910,18934,True
20948,14643,True
20949,19422,True
20950,11750,True
20951,7750,True
...,...,...
53091,15942,True
53093,19231,True
53203,0,True
53266,0,True


In [7]:
# Create new Impoverished column
df = cen_tract_city
cond_1 = df['MedianFamilyIncome'] <= 24250
df_below_income_threshold = df.loc[cond_1]
print(df_below_income_threshold.shape)
df_below_income_threshold.head(5)

(365, 147)


,GEOID,State,County,Urban,POP2010,OHU2010,GroupQuartersFlag,NUMGQTRS,PCTGQTRS,LILATracts_1And10,...,TractSeniors,TractWhite,TractBlack,TractAsian,TractNHOPI,TractAIAN,TractOMultir,TractHispanic,TractHUNV,TractSNAP
20910,17003957900,Illinois,Alexander,1,1610,718,0,107,0.066460,0,...,267,373,1187,5,1,4,40,4,160,293
20948,17019000301,Illinois,Champaign,1,4819,1970,0,165,0.034239,0,...,88,3617,198,710,2,8,284,381,731,40
20949,17019000302,Illinois,Champaign,1,3372,1971,0,0,0.000000,0,...,19,1696,180,1286,0,7,203,208,823,19
20950,17019000401,Illinois,Champaign,1,4843,539,1,3684,0.760685,0,...,4,3800,113,731,5,9,185,276,154,5
20951,17019000402,Illinois,Champaign,1,3931,1294,0,691,0.175782,0,...,3,3222,136,358,3,8,204,302,444,11


In [8]:
df_below_income_threshold['MedianFamilyIncome']

20910    18934
20948    14643
20949    19422
20950    11750
20951     7750
         ...  
53091    15942
53093    19231
53203        0
53266        0
53296        0
Name: MedianFamilyIncome, Length: 365, dtype: int64

In [9]:
df1 = df_below_income_threshold
df2 = cen_tract_all_shapes_df

merged_df = pd.merge(df1, df2, on="GEOID",how="left")
print(merged_df.shape)
merged_df.head(50)

(3340, 151)


,GEOID,State,County,Urban,POP2010,OHU2010,GroupQuartersFlag,NUMGQTRS,PCTGQTRS,LILATracts_1And10,...,TractNHOPI,TractAIAN,TractOMultir,TractHispanic,TractHUNV,TractSNAP,PointOrder,SubPolygonId,PointLatitude,PointLongitude
0,17003957900,Illinois,Alexander,1,1610,718,0,107,0.066460,0,...,1,4,40,4,160,293,1,1,37.0001,-89.1616
1,17003957900,Illinois,Alexander,1,1610,718,0,107,0.066460,0,...,1,4,40,4,160,293,2,1,36.9821,-89.1329
2,17003957900,Illinois,Alexander,1,1610,718,0,107,0.066460,0,...,1,4,40,4,160,293,3,1,36.9703,-89.1700
3,17003957900,Illinois,Alexander,1,1610,718,0,107,0.066460,0,...,1,4,40,4,160,293,4,1,36.9735,-89.1855
4,17003957900,Illinois,Alexander,1,1610,718,0,107,0.066460,0,...,1,4,40,4,160,293,5,1,36.9800,-89.1921
5,17003957900,Illinois,Alexander,1,1610,718,0,107,0.066460,0,...,1,4,40,4,160,293,6,1,36.9898,-89.1950
6,17003957900,Illinois,Alexander,1,1610,718,0,107,0.066460,0,...,1,4,40,4,160,293,7,1,37.0050,-89.1965
7,17003957900,Illinois,Alexander,1,1610,718,0,107,0.066460,0,...,1,4,40,4,160,293,8,1,37.0087,-89.1852
8,17003957900,Illinois,Alexander,1,1610,718,0,107,0.066460,0,...,1,4,40,4,160,293,9,1,37.0054,-89.1833
9,17003957900,Illinois,Alexander,1,1610,718,0,107,0.066460,0,...,1,4,40,4,160,293,10,1,37.0094,-89.1749


In [10]:
merged_df.to_csv('impoverished_'+State+'.csv')

In [ ]:
# cen_tract_all_shapes_df => Atlas
# cen_tract               => food desert info

## change CensusTract to GEOID
new_columns = cen_tract.columns.values
new_columns[0] = 'GEOID'
cen_tract.columns  = new_columns

## Step 1: import lookup table csv as a dataframe

In [2]:

desc_df = pd.read_csv('food_access_variable_lookup.csv')
desc_df.head(5)

,Field,LongName,Description
0,CensusTract,Census tract,Census tract number
1,State,State,State name
2,County,County,County name
3,Urban,Urban tract,Flag for urban tract
4,POP2010,"Population, tract total",Population count from 2010 census


## Step 2: remove any absolute data, we only want to keep information that is in percentage
The LongName column is of type string and we want to not inclue the row if the string ends with 'number'

In [3]:
df = desc_df
df_new = df.loc[~df['LongName'].str.endswith('number')]
print(df_new.shape)
df_share = df_new
df_share.head(5)

(76, 3)


,Field,LongName,Description
0,CensusTract,Census tract,Census tract number
1,State,State,State name
2,County,County,County name
3,Urban,Urban tract,Flag for urban tract
4,POP2010,"Population, tract total",Population count from 2010 census


## Step 3: Remove Descriptions that start with Flag, these are binary classifications and not populatoin data

In [4]:
df = df_share
df_new = df.loc[~df['Description'].str.startswith('Flag')]
print(df_new.shape)
df_share_noflag = df_new
df_share_noflag.head(5)

(60, 3)


,Field,LongName,Description
0,CensusTract,Census tract,Census tract number
1,State,State,State name
2,County,County,County name
4,POP2010,"Population, tract total",Population count from 2010 census
5,OHU2010,"Housing units, total",Occupied housing unit count from 2010 census


## Step 4: Keep only 1 out of the four distance categories in the Field column.  Save to csv to examine.

In [5]:
df = df_share_noflag
cond_1 = ~df['Field'].str.endswith('halfshare')
cond_2 = ~df['Field'].str.endswith('1share')
cond_3 = ~df['Field'].str.endswith('10share')
cond_4 = ~df['Field'].str.endswith('20share')
df_new = df.loc[cond_1 & cond_3 & cond_4]
print(df_new.shape)
df_share_noflag_simple = df_new[['Field','Description']]
df_share_noflag_simple.to_csv('short_description.csv')


(21, 3)
